In [1]:
from tavily import TavilyClient
import asyncio, os, requests, time, json
from IPython.display import display, Markdown, Latex

tavily_client = TavilyClient(api_key="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")

In [2]:
from openai import OpenAI
import math
import time
import json

client = OpenAI(
    base_url="http://34.67.3.74:8877/v1",
    api_key="token-abc123",
)

In [3]:
def deduplicate_and_format_sources(search_response, max_tokens_per_source, include_raw_content=True):
     # Collect all results
    sources_list = []
    for response in search_response:
        sources_list.extend(response['results'])
    
    # Deduplicate by URL
    unique_sources = {source['url']: source for source in sources_list}

    # Format output
    formatted_text = "Content from sources:\n"
    for i, source in enumerate(unique_sources.values(), 1):
        formatted_text += f"{'='*80}\n"  # Clear section separator
        formatted_text += f"Source: {source['title']}\n"
        formatted_text += f"{'-'*80}\n"  # Subsection separator
        formatted_text += f"URL: {source['url']}\n===\n"
        formatted_text += f"Most relevant content from source: {source['content']}\n===\n"
        if include_raw_content:
            # Using rough estimate of 4 characters per token
            char_limit = max_tokens_per_source * 4
            # Handle None raw_content
            raw_content = source.get('raw_content', '')
            if raw_content is None:
                raw_content = ''
                print(f"Warning: No raw_content found for source {source['url']}")
            if len(raw_content) > char_limit:
                raw_content = raw_content[:char_limit] + "... [truncated]"
            formatted_text += f"Full source content limited to {max_tokens_per_source} tokens: {raw_content}\n\n"
        formatted_text += f"{'='*80}\n\n" # End section separator
                
    return formatted_text.strip()

In [4]:
def generate_response(message_list):
    completion = client.chat.completions.create(
        model = "Llama-3.2-3B-Instruct",
        messages = message_list,
        max_tokens=2048,
        frequency_penalty=0.3,
        temperature=0.6,
        stream=True,
    )
    
    final_answer = []
    assistant_response = ""
    
    start = time.time()
    
    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in completion:
        chunk_content = chunk.choices[0].delta.content
        
        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
            assistant_response += chunk_content
    
    end = time.time()
    print(f"\n\ninference time: {end - start:.5f} sec \n\n")
    return assistant_response

In [5]:
import threading

def worker(query, search_result, req_num_result, include_raw, req_topic):
    print(f"Thread: {query}")
    search_result.append(
        tavily_client.search(
            query,
            max_results= req_num_result,
            include_raw_content= include_raw,
            topic= req_topic
        )
    )

In [6]:
def ask_tavily(search_queries, search_tasks, req_num_result, include_raw, req_topic):    
    start_time = time.time()
    threads = []
    
    for query in search_queries:
        t = threading.Thread(target=worker, args=(query, search_tasks, req_num_result, include_raw, req_topic))
        threads.append(t)
        t.start()

    for thread in threads:
        thread.join()
    
    end_time = time.time()
    execution_time = end_time - start_time

    print(f"\nask_tavily task running time: {execution_time:.2f}초 \n")

In [12]:
def ask_plan_query_writer(topic, content):
    llm_prompt = """You are an expert technical writer crafting a section that synthesizes information 
<section topic>
""" + topic + """
</section topic>

<section organization>
""" + content + """
</section organization>

<Task>
Your goal is to generate 3 web search queries that will help gather information for planning the sections. 

The queries should:

1. Be related to the section topic
2. Help satisfy the requirements specified in the section organization

Make the queries specific enough to find high-quality, relevant sources while covering the breadth needed for the section structure.

Note1. that today's date is """+time.strftime("%Y-%m-%d")+""".
Note2. Output your response in JSON format, with the following structure: { "queries": [ "query1", "query2", "query3" ] }
Only output in JSON format when generating responses. Never include additional phrases such as "here is content in JSON format".
</Task>"""

    return llm_prompt

In [8]:
def ask_final_writer_instructions(topic, content, search_tasks):
    final_section_writer="""You are an expert technical writer.

<Section name>
""" + content + """
</Section name>

<Section topic> 
""" + topic + """
</Section topic>

<Available Website Search Content>
""" + deduplicate_and_format_sources(search_tasks, max_tokens_per_source=4000, include_raw_content=True) + """
</Available Website Search Content>

<Task>
1. Section-Specific Approach:

For Introduction:
- Use # for Website Search title (Markdown format)
- Write in simple and clear language
- Focus on the core motivation for the Section in 1-2 paragraphs
- Use a clear narrative arc to introduce the Section
- Include NO structural elements (no lists or tables)
- No sources section needed

For Conclusion/Summary:
- Use ## for Conclusion/Summary title (Markdown format)
- For comparative Conclusion/Summary:
    * Must include a focused comparison table using Markdown table syntax
    * Table should distill insights from the Section
    * Keep table entries clear and concise
- For non-comparative Conclusion/Summary: 
    * Only use ONE structural element IF it helps distill the points made in the Section:
    * Either a focused table comparing items present in the Section (using Markdown table syntax)
    * Or a short list using proper Markdown list syntax:
      - Use `*` or `-` for unordered lists
      - Use `1.` for ordered lists
      - Ensure proper indentation and spacing
- Sources and url section needed. (especially when expressing a URL, please provide the entire URL exactly as given in the content without abbreviating it.)
- End with specific next steps or implications

2. Writing Approach:
- Use concrete details over general statements
- Make every word count
- Focus on your single most important point
</Task>

<Quality Checks>
- Verify that EVERY claim is grounded in the provided Source material
- Confirm each URL appears ONLY ONCE in the Source list
- For introduction: # for Website Search title, no structural elements, no sources section
- For conclusion: ## for Conclusion/Summary title, only ONE structural element at most, add sources and url section
- Markdown format
- Do not include word count or any preamble in your response
</Quality Checks>

Please note that respond in Korean always."""

    return final_section_writer

In [17]:
system_prompt = "You are a helpful assistant. "

topic = "판례확인"
content = "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"

messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": ask_plan_query_writer(topic, content)},
    ]

response_query = generate_response(messages)

{"queries": ["부하직원 비위사실 판결", "지방공무원법 부하직원 비위사실", "부하직원 비위사실 위법性 판결"]}

inference time: 1.41282 sec 




In [18]:
json_data = json.loads(response_query)
queries = json_data['queries']

print("사용자 발화 기반으로 추출한 web query 문장 3건:")
print(queries)

search_tasks = []
req_topic = 'general' # news   gerneral 과 news 중 선택
req_num_result = 3    # 각 web query 에 대해 리턴할 site 개수
include_raw = False    # site 의 원본 컨텐츠 리턴 유무

ask_tavily(queries, search_tasks, req_num_result, include_raw, req_topic)
print(search_tasks)

사용자 발화 기반으로 추출한 web query 문장 3건:
['부하직원 비위사실 판결', '지방공무원법 부하직원 비위사실', '부하직원 비위사실 위법性 판결']
Thread: 부하직원 비위사실 판결
Thread: 지방공무원법 부하직원 비위사실
Thread: 부하직원 비위사실 위법性 판결

ask_tavily task running time: 4.39초 

[{'query': '부하직원 비위사실 위법性 판결', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '상사는 부하직원 비위행위에 성실의무 책임 있는지? : 네이버 블로그', 'url': 'https://m.blog.naver.com/pdsph1004/222061319284', 'content': '22., 선고, 78누164, 판결] 지방공무원의 징계에 관한 어느 법규에도 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로 아무리 부하직원에게 파면에 해당하는 비위사실', 'score': 0.65146625, 'raw_content': None}, {'title': '대법원 2009다2545 - CaseNote - 케이스노트', 'url': 'https://casenote.kr/대법원/2009다2545', 'content': '선고 2009다2545 판결 [손해배상(기)] ... 대표이사가 해당 사업부서 등의 임원들에게 적절한 조치를 취하도록 지시하는 이상으로 부하직원들의 위법행위를 방지하기 위한 어떠한 구체적인 직무를 수행할 의무를 가진다고 보기 어렵다고 한 사례 ... 상급자들의 부당', 'score': 0.46337196, 'raw_content': None}, {'title': '정당한 징계사유가 존재하는지 여부 등이 문제된 사건[대법원 2021. 11. 25. 선고 중요판결] - 판례속보', 'url': 'https:/

In [21]:
search_tasks

[{'query': '부하직원 비위사실 위법性 판결',
  'follow_up_questions': None,
  'answer': None,
  'images': [],
  'results': [{'title': '상사는 부하직원 비위행위에 성실의무 책임 있는지? : 네이버 블로그',
    'url': 'https://m.blog.naver.com/pdsph1004/222061319284',
    'content': '22., 선고, 78누164, 판결] 지방공무원의 징계에 관한 어느 법규에도 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로 아무리 부하직원에게 파면에 해당하는 비위사실',
    'score': 0.65146625,
    'raw_content': None},
   {'title': '대법원 2009다2545 - CaseNote - 케이스노트',
    'url': 'https://casenote.kr/대법원/2009다2545',
    'content': '선고 2009다2545 판결 [손해배상(기)] ... 대표이사가 해당 사업부서 등의 임원들에게 적절한 조치를 취하도록 지시하는 이상으로 부하직원들의 위법행위를 방지하기 위한 어떠한 구체적인 직무를 수행할 의무를 가진다고 보기 어렵다고 한 사례 ... 상급자들의 부당',
    'score': 0.46337196,
    'raw_content': None},
   {'title': '정당한 징계사유가 존재하는지 여부 등이 문제된 사건[대법원 2021. 11. 25. 선고 중요판결] - 판례속보',
    'url': 'https://www.scourt.go.kr/portal/news/NewsViewAction.work?seqnum=8099&gubun=4',
    'content': '6. 25. 선고 2016두56042 판결 등 참조). 취업규칙은 노사 간의 집단적인 법률관계를 규정하는 법규범

In [31]:
print("\n\n=================================================================\n")
system_prompt = """You are a helpful assistant. And Answers must be in Korean. \
그리고 답변할땐 꼭 다음의 지시 사항을 준수해줘. \
1) 질문에 대한 답변 후보 문장들을 자세히 읽고 유저가 물어본 질문에 제시된 정보만 활용해서 질문과 정확성, 관련성, 신뢰성을 종합적으로 고려하여 답변을 만들어 제공해주세요. \
2) 후보문장은 json 형태로 돼 있습니다.
3) 후보문장 중 답변에 활용하는데 가장 적합한 후보가 어떤것 이였는지를 알려 주세요.\
4) 후보 문장들 중 질문에 대한 답이 없을 경우 "답변을 찾을 수 없습니다. 좀 더 구체적으로 질문해 주세요." 라고 답하세요."""

user_prompt =  content + "\n후보문장: " + str(search_tasks)

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

response_query = generate_response(messages)

print("\n\n=========================  Search Report  ========================================\n")
display(Markdown(response_query))




질문에 대한 답변 후보 문장들: 
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?

해당 질문에 대한 답변 후보 문장들: 
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"는 대법원 판결에 의하면, 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로, 아무리 부하직원에게 파면에 해당하는 비위사실이 있다 하더라도, 감독자는 위법을 인정하지 않는다.
  - 대법원 2009다2545 판결 등 참조. 

- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?는 대法원 판결에 의하면,부하직원이 징계에 해당하는 사안에 대하여는 수사기관의 비위 사실을 증명해야 하며, 그로 인해 상급자들의 부당한 징계가 발생할 수 있으므로, 정당한 징계사유가 존재하는지 여부가 문제된 사건이다.
  - 대法원 2021.11.25 선고 중요판결 등 참조.

해당 질문에 대한 답변 후보 문장들 중에서 가장 적합한 후보는: 

- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위 pháp을 인정하지 않는다."

inference time: 14.62169 sec 




=========================  Search Report  ========================================



질문에 대한 답변 후보 문장들: 
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?

해당 질문에 대한 답변 후보 문장들: 
- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?"는 대법원 판결에 의하면, 부하직원에게 파면에 해당하는 비위사실이 있는 경우에는 직접 감독 책임이 있는 공무원에게 감독 의무를 위반한 것으로 본다는 규정이 없으므로, 아무리 부하직원에게 파면에 해당하는 비위사실이 있다 하더라도, 감독자는 위법을 인정하지 않는다.
  - 대법원 2009다2545 판결 등 참조. 

- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가?는 대法원 판결에 의하면,부하직원이 징계에 해당하는 사안에 대하여는 수사기관의 비위 사실을 증명해야 하며, 그로 인해 상급자들의 부당한 징계가 발생할 수 있으므로, 정당한 징계사유가 존재하는지 여부가 문제된 사건이다.
  - 대法원 2021.11.25 선고 중요판결 등 참조.

해당 질문에 대한 답변 후보 문장들 중에서 가장 적합한 후보는: 

- 지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위법이 있는것인가? 
  - "지방공무원법상 부하직원의 비위사실이 있는 경우 감독자는 위 pháp을 인정하지 않는다."